In [163]:
pip install pandas numpy matplotlib seaborn scikit-learn xgboost

Note: you may need to restart the kernel to use updated packages.


In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler

df = pd.read_csv("C:/Users/Rouhan/python_projects/Delivery_Logistics/Delivery_Logistics.csv")

In [165]:
df.head()

,delivery_id,delivery_partner,package_type,vehicle_type,delivery_mode,region,weather_condition,distance_km,package_weight_kg,delivery_time_hours,expected_time_hours,delayed,delivery_status,delivery_rating,delivery_cost
0,250.99,delhivery,automobile parts,bike,same day,west,clear,297.0,46.96,1970-01-01 00:00:00.000000008,1970-01-01 00:00:00.000000008,no,delivered,3,1632.7206
1,250.99,xpressbees,cosmetics,ev van,express,central,cold,89.6,47.39,1970-01-01 00:00:00.000000002,1970-01-01 00:00:00.000000003,no,delivered,5,640.1700
2,250.99,shadowfax,groceries,truck,two day,east,rainy,273.5,26.89,1970-01-01 00:00:00.000000010,1970-01-01 00:00:00.000000016,no,delivered,4,1448.1700
3,250.99,dhl,electronics,ev van,same day,east,cold,269.7,12.69,1970-01-01 00:00:00.000000006,1970-01-01 00:00:00.000000008,no,delivered,3,1486.5700
4,250.99,dhl,clothing,van,two day,north,foggy,256.7,37.02,1970-01-01 00:00:00.000000009,1970-01-01 00:00:00.000000016,no,delivered,4,1394.5600


In [166]:
df.drop(columns = ['delivery_id','delivery_time_hours','expected_time_hours','delivery_rating','delayed'], axis=1, inplace=True)

In [167]:
ohe = OneHotEncoder(sparse_output=False)
onehotcolumns = ['vehicle_type', 'delivery_mode', 'weather_condition', 'region','delivery_status','package_type','delivery_partner']
ohe_data = ohe.fit_transform(df[onehotcolumns])
ohe_df = pd.DataFrame(ohe_data, columns=ohe.get_feature_names_out(onehotcolumns))
df = pd.concat([df.reset_index(drop=True), ohe_df.reset_index(drop=True)], axis=1)
df = df.drop(columns = onehotcolumns)
df.head()

,distance_km,package_weight_kg,delivery_cost,vehicle_type_bike,vehicle_type_ev bike,vehicle_type_ev van,vehicle_type_scooter,vehicle_type_truck,vehicle_type_van,delivery_mode_express,delivery_mode_same day,delivery_mode_standard,delivery_mode_two day,weather_condition_clear,weather_condition_cold,weather_condition_foggy,weather_condition_hot,weather_condition_rainy,weather_condition_stormy,region_central,region_east,region_north,region_south,region_west,delivery_status_delayed,delivery_status_delivered,delivery_status_failed,package_type_automobile parts,package_type_clothing,package_type_cosmetics,package_type_documents,package_type_electronics,package_type_fragile items,package_type_furniture,package_type_groceries,package_type_pharmacy,delivery_partner_amazon logistics,delivery_partner_blue dart,delivery_partner_delhivery,delivery_partner_dhl,delivery_partner_ecom express,delivery_partner_ekart,delivery_partner_fedex,delivery_partner_shadowfax,delivery_partner_xpressbees
0,297.0,46.96,1632.7206,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,89.6,47.39,640.1700,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,273.5,26.89,1448.1700,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,269.7,12.69,1486.5700,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,256.7,37.02,1394.5600,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [168]:

X = df.drop('delivery_cost', axis=1)
Y = df['delivery_cost']

In [169]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
p1 = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb_regressor', XGBRegressor(objective ='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)),
])
p1.fit(X_train, Y_train)
Y_pred = p1.predict(X_test)
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error: 3.494022243696288
Mean Squared Error: 19.6287852903244
R^2 Score: 0.99989558509738


In [170]:
import pickle
pickle.dump(p1, open("delivery_cost_model.pkl", "wb"))
pickle_model = pickle.load(open("delivery_cost_model.pkl", "rb"))

In [171]:
pickle_model.predict(X_test.iloc[0:5])

array([1258.7664 ,  709.72125,  810.08154,  829.59875,  998.169  ],
      dtype=float32)